In [1]:
import numpy as np
import torch 
from torch.utils.data import DataLoader, random_split
import torchvision
import torch.nn.functional as F
import torchvision.transforms as T
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid, save_image
import matplotlib.pyplot as plt
import os
import random
import copy
from collections import defaultdict

In [2]:
pip install torchsummary

Note: you may need to restart the kernel to use updated packages.


## 1. Data Preprocessing

In [3]:
def preprocessing(main_folder_path):
    # list of folders in the main folder
    listdir = sorted(os.listdir(main_folder_path))
    
    # Image Transformation
    mean = [0.485, 0.456, 0.406] # mean of image tensor
    std = [0.229, 0.224, 0.225]  # standard devisation of image tensor
    normalize_stat = (mean, std)
    size = 224

    transform = T.Compose([
        T.Resize(size),
        T.CenterCrop(size),
        T.ToTensor(),
        T.Normalize(*normalize_stat)
    ])

    dataset = ImageFolder(main_folder_path, transform=transform)
    classes = dataset.classes

    return dataset, classes

In [4]:
main_folder_path = '../input/pins-face-recognition/105_classes_pins_dataset'
dataset, classes = preprocessing(main_folder_path)

print("Total dataset size: ", len(dataset))
print("Number of classes: ", len(classes))

Total dataset size:  17534
Number of classes:  105


In [5]:
def dataloader(dataset, classes, train_batch_size, test_batch_size, num_clients):
    torch.manual_seed(42)
    
    # 70% to 30% train to test
    num_val = int(len(dataset) * 0.3)
    num_train = len(dataset) - num_val
    
    # 1. Train and Validation Data
    train_data, test_data = random_split(dataset, [num_train, num_val])
    
    # save train_data size (n)
    n = len(train_data)
    
    # Partition data for federated learning
    train_loader_list = []
    
    # nk / n list
    nk_n_list = []
    
    # Client ID: corresponding train_loader
    client_loader_dict = {}
    
    # Split train dataset
    total_train_size = len(train_data)
    examples_per_client = total_train_size // num_clients
    client_datasets = random_split(train_data, [min(i + examples_per_client, 
               total_train_size) - i for i in range(0, total_train_size, examples_per_client)])
    
    client_datasets = client_datasets[:num_clients]
    
    
    # Save dataloader for each
    for client_id, client_data in enumerate(client_datasets):
        
        # Save nk / n
        nk_n = len(client_data) / len(train_data)
        nk_n_list.append(nk_n)
        
        train_loader = DataLoader(client_data, batch_size=train_batch_size, shuffle=True, pin_memory=False, num_workers=3)
        train_loader_list.append(train_loader)
        client_loader_dict[client_id] = train_loader
    
    # Train and Validation Dataloader
    loader_idx = 0
    for key, _ in client_loader_dict.items():
        client_loader_dict[key] = train_loader_list[loader_idx]
        loader_idx += 1
    
    test_loader = DataLoader(test_data, batch_size=test_batch_size, shuffle=False, pin_memory=False, num_workers=3)
    
    return train_loader_list, client_loader_dict, test_loader, test_data, nk_n_list

In [6]:
num_clients = 5
train_batch_size = 64
test_batch_size = 32

train_loader_list, client_loader_dict, test_loader, test_data, nk_n_list = dataloader(dataset, classes, train_batch_size, test_batch_size, num_clients)

for i in nk_n_list:
    print(i)

0.19993482157405898
0.19993482157405898
0.19993482157405898
0.19993482157405898
0.19993482157405898


/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## Model

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchsummary import summary
from glob import glob
from tqdm.notebook import tqdm

In [8]:
class black_box_model(nn.Module):
    def __init__(self, num_classes, pretrained=True):
        super().__init__()
        # Use a pretrained model
        self.resnet34 = models.resnet34(True)
        self.features = nn.Sequential(*list(self.resnet34.children())[:-1])
        # Replace last layer
        self.classifier = nn.Sequential(nn.Flatten(),
                                         nn.Linear(self.resnet34.fc.in_features, num_classes))

    def forward(self, x):
        x = self.features(x)
        y = self.classifier(x)
        return y
    
    def summary(self, input_size):
        return summary(self, input_size)

In [9]:
main_folder_path = '../input/pins-face-recognition/105_classes_pins_dataset'
dataset, classes = preprocessing(main_folder_path)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_classes = len(classes)

global_model = black_box_model(num_classes=num_classes).to(device)
# global_model.summary((3, 224, 224))

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

## 3. Federated Learning

In [10]:
# # Federated Averaging Algorithm (Non-IID)
# def fedAvg(w, nk_n_list, client_id_selected):
    
#     print("")
#     print("Averaging client_id list: ", client_id_selected)
    
#     first_id = client_id_selected[0]
#     w_avg = copy.deepcopy(w[first_id])
    
#     # Update first weight parameter
#     for k in w_avg.keys():
#         nk_n = nk_n_list[first_id]
#         w_avg[k] = w_avg[k] + (nk_n * w[first_id][k])
    
#     # Remove the updated id weight
#     client_id_selected.remove(first_id)
    
#     print("After removing first index: ", client_id_selected)
#     print("")
    
#     # Use selected client's weight parameters
#     for k in w_avg.keys():
#         for i in client_id_selected: 
#             nk_n = nk_n_list[i]
#             w_avg[k] = w_avg[k] + (nk_n * w[i][k])
        
#         w_avg[k] = torch.div(w_avg[k], len(w))
    
#     return w_avg

In [36]:
from collections import OrderedDict

def fedAvg(client_model_list):
    new_params = OrderedDict()
    
    n = len(client_model_list)  # number of clients
    
    for client_model in client_model_list:
        local_param = client_model.state_dict()  # get current parameters of one client
        for k, v in local_param.items():
            new_params[k] = new_params.get(k, 0) + v / n
    
    return new_params

## 4. Train and Evaluate

In [54]:
import torch
import torch.nn as nn
from tqdm.notebook import tqdm

# Code Referance: https://www.kaggle.com/code/pezhmansamadi/facerecognition-torch-resnet34

def local_train(e, local_epochs, local_model, train_loader, device, criterion, optimizer):
    torch.cuda.empty_cache()
    local_model.train(True)
    torch.set_grad_enabled(True)
    
    total = 0
    correct = 0
    local_loss = 0.0
    local_acc = 0
    
    tq_batch = tqdm(train_loader, total=len(train_loader))
    for images, labels in tq_batch:
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outs = local_model(images)
        _, preds = torch.max(outs, 1)
        
        # outs = torch.exp(outs)
        loss = criterion(outs, labels)
        loss.backward()
        optimizer.step()
        local_loss += loss.item()
        
        total += labels.size(0)
        correct += (preds == labels).sum()
        local_acc += float(correct) / total
        
        tq_batch.set_description(f'Local Epoch [{e + 1}/{local_epochs}]')
        tq_batch.set_postfix_str('Local loss = {:.4f} ; Local acc = {:.4f} '.format(loss.item(), float(correct) / total))
    
    # Average loss and acc of the training batch
    local_acc = local_acc / len(train_loader)
    local_loss = local_loss / len(train_loader)
    local_w = local_model.state_dict()

    return local_model, local_loss, local_w, local_acc

def global_evaluate(global_model, test_loader, criterion, device):
    torch.cuda.empty_cache()
    global_model.eval()
    
    total = 0
    glob_loss = 0
    glob_acc = 0
    correct = 0
    
    tq_batch = tqdm(test_loader, total=len(test_loader), leave=False)
    for images, labels in tq_batch:
        images = images.to(device)
        labels = labels.to(device)

        outputs = global_model(images)
        loss = criterion(outputs, labels)
        glob_loss += loss.item()
        
        _, preds = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (preds == labels).sum()
        
        glob_acc += float(correct) / total
        batch_acc = float(correct) / total
        
        tq_batch.set_postfix_str('Global loss = {:.4f} ; Global acc = {:.4f} '.format(loss.item(), batch_acc))
    
    # Average global loss and acc of the batch
    glob_loss = glob_loss / len(test_loader)
    glob_acc = glob_acc / len(test_loader)

    return glob_acc, glob_loss

In [55]:
def federated_learning(num_selected, num_rounds, num_clients, num_local_epochs, global_model, client_model_list, client_loader_dict, device):
    torch.cuda.empty_cache()
    
    id_loss_dict = defaultdict(list)
    
    local_w_list = [None for i in range(num_clients)]
    local_loss_list = [100 for i in range(num_clients)]
    
    glob_acc_list = [0 for i in range(num_rounds)]
    glob_loss_list = [0 for i in range(num_rounds)]
    
    criterion = nn.CrossEntropyLoss()
    
    # For each round
    for each_round in tqdm(range(num_rounds)):
        
        # Randomly selected client id "EACH ROUND"
        num_client_list = [i for i in range(num_clients)]
        client_id_selected = num_client_list
        # client_id_selected = random.sample(num_client_list, k=num_selected)
        print("Selected client_id: ", client_id_selected)
        
        # For each client
        for client_id in tqdm(client_id_selected):
            print("")
            print(f"Updating [client_id]: {client_id+1}")
            print("")
            local_model = client_model_list[client_id]
            local_dataloader = client_loader_dict[client_id]
            
            learning_rate = 0.05
            optimizer = torch.optim.SGD(local_model.parameters(), lr=learning_rate)
            
            # For each local epoch for each client
            for e in tqdm(range(num_local_epochs)):
                # Train local client models
                local_updated_model, new_local_loss, local_w, local_acc = local_train(e, num_local_epochs, local_model, local_dataloader, device, criterion, optimizer)
                current_loss = local_loss_list[client_id]
                
                # Save loss for each client id
                id_loss_dict[client_id].append(new_local_loss)
                
                # Append new local_loss and local_weight
                local_loss_list[client_id] = new_local_loss
                local_w_list[client_id] = local_w 
                
                print(f"[rounds]: {each_round + 1}/{num_rounds} - [client_id]: {client_id + 1}/{num_clients} - [local_epoch]: {e+1}/{num_local_epochs} - [local_loss]: {new_local_loss} - [local_acc]: {local_acc*100}%")
                
        # Federaed Averaging
        # new_glob_w = fedAvg(local_w_list, nk_n_list, client_id_selected)
        new_glob_w = fedAvg(client_model_list)
        global_model.load_state_dict(new_glob_w)
        
        # Send new global model back to clients
        print("")
        print("Sending global model weight to local client models...")
        print("")
        for loc_model in client_model_list:
            loc_model.load_state_dict(new_glob_w)
        
        # Evaluate Global Model
        glob_acc, glob_loss = global_evaluate(global_model, test_loader, criterion, device)
        glob_acc_list.append(glob_acc)
        glob_loss_list.append(glob_loss)
        
        print("")
        print("*"*100)
        print("")
        print(f"[rounds]: {each_round + 1}/{num_rounds} - [global_loss]: {glob_loss} - [global_acc]: {glob_acc*100}%")
        print("")
        print("*"*100)
        print("")
        
    return glob_acc_list, glob_loss_list, id_loss_dict

In [56]:
main_folder_path = '../input/pins-face-recognition/105_classes_pins_dataset'
   
num_clients = 5
train_batch_size = 64
test_batch_size = 32

dataset, classes = preprocessing(main_folder_path=main_folder_path)
train_loader_list, client_loader_dict, test_loader, _, nk_n_list = dataloader(dataset, classes, train_batch_size, test_batch_size, num_clients)

for idx, dt in client_loader_dict.items():
    print(f'Client {idx} dataloader size: {len(dt)}')

print("")
print("Test dataloader size: ", len(test_loader))

Client 0 dataloader size: 39
Client 1 dataloader size: 39
Client 2 dataloader size: 39
Client 3 dataloader size: 39
Client 4 dataloader size: 39

Test dataloader size:  165


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
global_model = black_box_model(num_classes=num_classes).to(device)
client_model_list = [black_box_model(num_classes=num_classes).to(device) for _ in range(num_clients)]

# num_selected = num_clients only for iid setting
num_clients = 5
num_selected = 5
num_rounds = 10
num_local_epochs = 3

glob_acc_list, glob_loss_list, id_loss_dict = federated_learning(num_selected,
                                                                 num_rounds, 
                                                                 num_clients, 
                                                                 num_local_epochs, 
                                                                 global_model, 
                                                                 client_model_list, 
                                                                 client_loader_dict, 
                                                                 device)

  0%|          | 0/10 [00:00<?, ?it/s]

Selected client_id:  [0, 1, 2, 3, 4]


  0%|          | 0/5 [00:00<?, ?it/s]


Updating [client_id]: 0



  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/39 [00:00<?, ?it/s]

[rounds]: 10/10 - [client_id]: 5/5 - [local_epoch]: 2/3 - [local_loss]: 0.014546990943833804 - [local_acc]: 100.0%


  0%|          | 0/39 [00:00<?, ?it/s]

[rounds]: 10/10 - [client_id]: 5/5 - [local_epoch]: 3/3 - [local_loss]: 0.009427364832029128 - [local_acc]: 100.0%

Sending global model weight to local client models...



  0%|          | 0/165 [00:00<?, ?it/s]

In [ ]:
import matplotlib.pyplot as plt

num_round_list = list(range(1, num_rounds+1))

plt.figure(figsize=(7, 7))
plt.grid(axis='y')
plt.title("Robust Accuracy after Adversarial Training (FGSM(eps=0))")
plt.xlabel("Iteration")
plt.ylabel("Robust Accuracy(%)")

for eps in epsilons:
    plt.plot(iteration, eps_accuracy_dict[eps], label=("eps={}".format(eps)))
    plt.legend()

glob_acc_list, glob_loss_list, id_loss_dict